In [8]:
import torch
import torch.nn as nn      # Main imports
from torch.nn import functional as F    #nead embeding for "Convolution" of all elements in a tensor essentially mean for convol operation



In [9]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 50000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU over CPU for traning
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(123)    #For init of weights in a same random manner

In [10]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()    
#Download and read of a dataset tinyshakespeare

--2023-02-07 10:00:13--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2023-02-07 10:00:13 (153 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [11]:
# here are all the unique characters that happened to be in this text
chars = sorted(list(set(text)))
print("All characters in  a text: {}",chars)
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
#print(stoi)
itos = { i:ch for i,ch in enumerate(chars) }
#print(itos)
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers

decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

All characters in  a text: {} ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [12]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data)) # first 80% will be train, rest validation
train_data = data[:n]
val_data = data[n:]

In [13]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

        

In [16]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


In [17]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.6162, val loss 4.6161
step 500: train loss 4.3983, val loss 4.4021
step 1000: train loss 4.1959, val loss 4.2030
step 1500: train loss 4.0084, val loss 4.0183
step 2000: train loss 3.8356, val loss 3.8471
step 2500: train loss 3.6773, val loss 3.6909
step 3000: train loss 3.5315, val loss 3.5489
step 3500: train loss 3.3989, val loss 3.4196
step 4000: train loss 3.2799, val loss 3.3016
step 4500: train loss 3.1720, val loss 3.1965
step 5000: train loss 3.0756, val loss 3.1014
step 5500: train loss 2.9898, val loss 3.0177
step 6000: train loss 2.9145, val loss 2.9450
step 6500: train loss 2.8476, val loss 2.8803
step 7000: train loss 2.7895, val loss 2.8218
step 7500: train loss 2.7387, val loss 2.7734
step 8000: train loss 2.6950, val loss 2.7301
step 8500: train loss 2.6565, val loss 2.6933
step 9000: train loss 2.6226, val loss 2.6630
step 9500: train loss 2.5963, val loss 2.6366
step 10000: train loss 2.5725, val loss 2.6137
step 10500: train loss 2.5515, val lo

In [18]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


IESCI lly Co, sig haimor o. merit.
bl outan RKI:
THARG je,
TUCE he? we wenathse.

BUCUL:
Thendgn hedil cor s thecath,
Hill'dathureenos mon s t weo, n dis?
Anhmyoowanevetig aindanous we ce y:

angrad?

S d thefour fomy. G es?
ADin ys aritheave anorkithyou gn INGoor.
DWhes becy eerexpacat she hisoled gnodrth p tinghe'tilis thome t d.
QUSow lawope dvee, n

An tilorous fimy,
LK:
wisigang, hereld ay, toa hitised o bus;
TEDomat alolougr: mathalimourer.
Andarofort n e;
QULA:
S:
HAstsarr, womun lity
Sou
